In [ ]:
import os
IN_KAGGLE = 'KAGGLE_URL_BASE' in os.environ
IN_KAGGLE

In [ ]:
if IN_KAGGLE:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    secret_value = user_secrets.get_secret("gittoken2")
    !git clone https://{secret_value}@github.com/moienr/CropMapping.git

In [ ]:
if IN_KAGGLE:
    import time
    import os
    sleep_time = 5
    while not os.path.exists("/kaggle/working/CropMapping"):
        print("didn't find the path, wating {sleep_time} more seconds...")
        time.sleep(sleep_time)
    print("path found...")
    import sys
    sys.path.append("/kaggle/working/CropMapping")
    sys.path.append("/kaggle/working/CropMapping/dataset")

In [ ]:
%%capture
if IN_KAGGLE:
    !pip install geemap
    !pip install geedim

In [ ]:
import ee
import geemap
import importlib
import ast
import pandas as pd

In [ ]:
# Use the service account credentials to authenticate.
service_account = 'kaggle@ee-mrangzan.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, '/kaggle/working/CropMapping/ee-mrangzan-e0965f83960a.json')

# Initialize the Earth Engine client.
ee.Initialize(credentials)

In [ ]:
Map = geemap.Map()

In [ ]:
import utils.ee_utils as ee_utils

In [ ]:
# Reload the module
importlib.reload(ee_utils)

In [ ]:
date1 = '2018-8-01'
date2 = '2018-8-30'

In [ ]:

# Define area of interest.
# roi = ee.Geometry.Polygon(
#                                 [[[1.2897224677359675, 48.38400739957012],
#                                 [1.2897224677359675, 47.93697958008443],
#                                 [3.2068367255484675, 47.93697958008443],
#                                 [3.2068367255484675, 48.38400739957012]]])
roi = ee.Geometry.Polygon( [[[2.220233, 48.116767],
                             [2.301807, 48.116767],
                             [2.301807, 48.167835],
                             [2.220233, 48.167835],
                             [2.220233, 48.116767]]])


# Call function with defined start and end dates and area of interest.
cloudless_collection = ee_utils.create_s2_collection(roi, date1, date2)

# Define visualization parameters.
visualization = {
        'min': 0.0,
        'max': 0.3,
        'bands': ['B4', 'B3', 'B2'],
}

# Add layer to map.
Map.addLayer(cloudless_collection.median().clip(roi), visualization, 'Cloudless Collection')
Map.centerObject(roi, 10)

In [ ]:
s1_mosaic = ee_utils.get_combined_s1_image(roi, date1, date2)
s1_mosaic = ee_utils.sen1_add_ratio(s1_mosaic, vv_name='VV', vh_name='VH', ratio_name='ratio')

false_color = {
    'bands': ['VV', 'VH', 'ratio'],
    'min': [-25, -35, 0],
    'max': [0, -5, 10]
}
Map.addLayer(s1_mosaic, false_color, 'S1 Mosaic')
Map.centerObject(roi, 10)

In [ ]:
crop_map = ee.Image('JRC/D5/EUCROPMAP/V1/2018')
# binari mask for crop map for mazie 216
crop_map = crop_map.eq(216)

# Replace null values with 0
crop_map = crop_map.unmask(0).clip(roi)

Map.addLayer(crop_map, {}, 'Crop Map')

In [ ]:
s1_mosaic

In [ ]:
Map

In [ ]:
# get roi from polygon geemap
# roi = Map.draw_last_feature.geometry()
# print(roi)

In [ ]:
print(roi.getInfo())

In [ ]:
sen1_short_print = lambda s2_collection: ee_utils.ee_property_printer(s2_collection, propertie_name_list=['system:time_start','platform_number',
                                                                                           'system:band_names','instrumentMode',                         
                                                                                            'relativeOrbitNumber_stop','relativeOrbitNumber_start',
                                                                                            'cycleNumber'],
                                                             first_is_t_in_millis=True, df_instead_of_print=False)

In [ ]:
def download_s1_s2(roi, start_date, end_date, s1_folder="", s2_folder="",
                   s1_file_name="s1_56-48.tif", s2_file_name="s2_56-48.tif",
                   scale=10, bands=['B2', 'B3', 'B4'], proj='EPSG:3857'):
    print("downloading s1 ...")
    s1_img = ee_utils.get_combined_s1_image(roi, start_date, end_date)
    print("downloading s2 ...")
    s2_img = ee_utils.get_combined_s2_image(roi, start_date, end_date)
    s2_img = s2_img.select(bands)

    # EPSG:4326 or EPSG:3857
    geemap.download_ee_image(s1_img, s1_folder + s1_file_name, crs=proj, scale=scale, region=roi)
    geemap.download_ee_image(s2_img, s2_folder + s2_file_name, crs=proj, scale=scale, region=roi)

In [ ]:
# download_s1_s2(roi, '2019-05-01', '2019-06-30', scale=10, bands=['B2','B3','B4'])

In [ ]:
# crop_map = ee.Image('JRC/D5/EUCROPMAP/V1/2018').clip(roi)
# geemap.download_ee_image(crop_map, "crop_map"+".tif",crs='EPSG:3857', scale=10,region = roi)

In [ ]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
def get_previous_months(harvest_date):
    # Convert harvest date string to datetime object
    harvest_datetime = datetime.strptime(harvest_date, '%Y-%m-%d')
    
    # Create a list to store the previous months
    previous_months = []
    
    # Loop through the previous six months and add them to the list
    for i in range(1, 7):
        previous_month = ((harvest_datetime - relativedelta(months=i+1)).strftime('%Y-%m-%d'), (harvest_datetime - relativedelta(months=i)).strftime('%Y-%m-%d'))
        
        previous_months.append(previous_month)
        
    previous_months.reverse()
    return previous_months

dummy_harvest_date = '2019-08-01'
previous_months = get_previous_months(dummy_harvest_date)
print(previous_months)

In [ ]:
def name_from_roi(roi):
    center = roi.centroid().getInfo()['coordinates']
    x = str(center[0]).replace('.','-')[0:5]
    y = str(center[1]).replace('.','-')[0:5]
    return x + "_" + y + ".tif"

name_from_roi(roi)

In [ ]:
import os
def check_folder_exists(folder):
    if not os.path.exists(folder):
        print("Creating folder: " + folder)
        os.makedirs(folder)

In [ ]:
def ts_downloader(roi, harvest_date, scale=10, bands=['B2','B3','B4'], proj='EPSG:3857', dataset_folder="ts_dataset"):
    ts_dates = get_previous_months(harvest_date)
    name = name_from_roi(roi)
    for i, ts_date in enumerate(ts_dates):
        print(f"Downloading Date {i+1}: {ts_date[0]} to {ts_date[1]}")
        check_folder_exists(dataset_folder + "//s1//" + str(i+1))
        check_folder_exists(dataset_folder + "//s2//" + str(i+1))
        download_s1_s2(roi, ts_date[0], ts_date[1], s1_folder=dataset_folder + "//s1//" + str(i+1) + "//",
                                                    s2_folder=dataset_folder + "//s2//" + str(i+1) + "//",
                                                    s1_file_name=name,  s2_file_name=name,
                                                    scale=scale, bands=bands)
    
    print("Downloading Crop Map...")
    crop_map = ee.Image('JRC/D5/EUCROPMAP/V1/2018').clip(roi)
    # none vlue to 0
    crop_map = crop_map.unmask(0)
    check_folder_exists(dataset_folder + "//" + "crop_map//")
    geemap.download_ee_image(crop_map, dataset_folder + "//" + "crop_map//" + name, crs='EPSG:3857', scale=10,region = roi)
    
    print("Done!")

In [ ]:
if IN_KAGGLE:
    rois_path = '/kaggle/working/CropMapping/dataset/Europe.xlsx'
else:
    rois_path = 'D:\\python\\CropMapping\\dataset\\Europe.xlsx'

In [ ]:
df = pd.read_excel(rois_path)

In [ ]:
BANDS = ['B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12']

In [ ]:
if IN_KAGGLE:
    DATASET_FOLDER = "../../tmp/ts_dataset"
else:
    DATASET_FOLDER = "ts_dataset"

In [ ]:
COUNTRY = 'france_valid'

In [ ]:
for index, row in df.iterrows():
    if index ==0 or row['country'] != COUNTRY:
        continue
    print(f"Processing {index+1} of {len(df)} - {row['country']}")
    r_roi = row['roi']
    r_roi = r_roi.replace('\n', '').replace(' ', '')
    r_roi = ast.literal_eval(r_roi)
    r_roi = ee.Geometry.Polygon(r_roi)
    r_date = row['harvest_date'] 

    ts_downloader(r_roi, r_date, scale=10, bands=BANDS, dataset_folder=DATASET_FOLDER)

In [ ]:
from utils import utils
importlib.reload(utils)

In [ ]:
os.listdir(DATASET_FOLDER)

In [ ]:
if IN_KAGGLE:
    !du -sh ../../tmp/ts_dataset

In [ ]:
# ts_dataset
# |-- crop_map
# |-- s1
#     |-- 1
#     |-- 2
#     |-- 3
#     |-- 4
#     |-- 5
#     |-- 6
# |-- s2
#     |-- 1
#     |-- 2
#     |-- 3
#     |-- 4
#     |-- 5
#     |-- 6

# Loop through the folders in the dataset


# Patching

In [ ]:
main_in_folder = "../../tmp/ts_dataset//" if IN_KAGGLE else "ts_dataset//"
main_out_folder = "../../tmp/ts_dataset_patched//" if IN_KAGGLE else "ts_dataset_patched//"
check_folder_exists(main_out_folder)

In [ ]:
# finding path available volumes
# volumes = !ls /kaggle/input


In [ ]:

folders = ["s1", "s2"]
for folder in folders:
    ts_date_folders = os.listdir(main_in_folder + folder)
    for date in ts_date_folders:
        print("Processing: " + main_in_folder + folder + "//" + date)
        input_sat = "S2" if folder == "s2" else "S1"
        utils.patch_folder(main_in_folder + folder + "//" + date+ "//", main_out_folder + folder + "//" + date+ "//", input_sat=input_sat, remove_year=False,mute=True, no_overlap=True)

# patch crop map
print("Processing: " + main_in_folder + "crop_map" + "//")
utils.patch_folder(main_in_folder + "crop_map" + "//", main_out_folder + "crop_map" + "//", input_sat="crop_map", remove_year=False,mute=True, no_overlap=True)


In [ ]:
if IN_KAGGLE:
    !du -sh ../../tmp/ts_dataset_patched//

In [ ]:
NAME = "crop_map_dataset_" + COUNTRY
NAME

In [ ]:
# zip main_out_folder to kaggle/working/crop_map_dataset.zip
if IN_KAGGLE:
    import shutil
    shutil.make_archive("../working/" + NAME, 'zip', main_out_folder)

In [ ]:
if IN_KAGGLE:
    !du -sh ../working/

In [ ]:
# delete extra files
if IN_KAGGLE:
    !rm -r ../working/CropMapping